In [1]:
import os
from pathlib import Path
from subprocess import Popen
import boto3
import zipfile

In [2]:
s3 = boto3.client('s3')

In [7]:
download_bucket = "temp"
upload_bucket = "output"

TMPDIR = Path("zillow-data-raw/20191001/")
FIPS_table = ['2']

In [8]:
FipsFile = FIPS_table[0] + '.zip'
Key = '20191001/' + FipsFile
Filename = str(TMPDIR / FipsFile)
print(Filename)

zillow-data-raw/20191001/2.zip


In [9]:
# download zip file from s3 to ec2
s3.download_file(
    Bucket=download_bucket,
    Key=Key,
    Filename=Filename
)

ClientError: An error occurred (403) when calling the HeadObject operation: Forbidden